#### Imports

In [67]:
import pandas as pd
import sys, importlib

# Option 1. Append absolute path to folder holding api_wrapper
absolute_path_to_api_wrapper = "/Users/joshhellings/Documents/ECO/RADataHub/ChartOfTheDay/templates/"
sys.path.append(absolute_path_to_api_wrapper)
import api_wrapper.api_hub as api_hub

# # Option 2. Append relative path
# sys.path.append('../api_wrapper/')
# from api_hub import EconDataAPI

---

### Fed Interest Rate

Series ID: `DFF`

Federal Funds Effective Rate

In [71]:
# Reload module & instantiate API wrapper
importlib.reload(api_hub)
api = api_hub.EconDataAPI()

pd.DataFrame(api.get_fred_series('fed_rate'))

name        id     frequency  \
0              Federal Funds Effective Rate       DFF  Daily, 7-Day   
1              Federal Funds Effective Rate  FEDFUNDS       Monthly   
2  Federal Funds Target Range - Upper Limit  DFEDTARU  Daily, 7-Day   

                              units  
0  Percent, Not Seasonally Adjusted  
1  Percent, Not Seasonally Adjusted  
2  Percent, Not Seasonally Adjusted

Effective Rates

In [81]:
# # Get daily series, effective rate
# df_d = api.get_fred_data(series='DFF')

# # Get monthly series, effective rate
# df_m = api.get_fred_data(series='FEDFUNDS')

Target Rates

In [84]:
# Get daily series, target rate, upper limit
df_t = api.get_fred_data(series='DFEDTARU')

# prior to December 16, 2008, Fed rate was reported as a single target rate
df_t_pre08 = api.get_fred_data(series='DFEDTAR')

# Concatenate dataframes
df_t = pd.concat([df_t_pre08, df_t])

In [85]:
df_t

date  value  year
0    1982-09-27  10.25  1982
1    1982-09-28  10.25  1982
2    1982-09-29  10.25  1982
3    1982-09-30  10.25  1982
4    1982-10-01  10.00  1982
...         ...    ...   ...
5429 2023-10-28   5.50  2023
5430 2023-10-29   5.50  2023
5431 2023-10-30   5.50  2023
5432 2023-10-31   5.50  2023
5433 2023-11-01   5.50  2023

[15011 rows x 3 columns]

---

### Chart

In [74]:
import altair_wrapper.altair_eco

In [19]:
def reload_altair_wrapper(df: pd.DataFrame):
    """Reload import of CustomChart class from altair_wrapper"""
    importlib.reload(altair_wrapper.altair_eco)
    eco = altair_wrapper.altair_eco.CustomChart(df)
    return eco

In [93]:
importlib.reload(altair_wrapper.altair_eco)
eco = altair_wrapper.altair_eco.CustomChart(df_t, theme='dark', start_date='2007-01-01')

custom_chart = eco.create_line_chart(
    x='date',
    y='value',
    title="Fed Interest Rate, Nov 2023",
    titleDx=28,
    subtitle=["Federal funds target range, upper limit, recessions highlighted", "Source: FRED"],
    interpolate='step-after',
    add_recessions=True,
)

custom_chart.display()

alt.LayerChart(...)

Save chart

In [94]:
from datetime import date
# Get today's date in formats YYYYMM & YYYYMMDD
yearmonth = date.today().strftime("%Y%m")
today = date.today().strftime("%Y%m%d")

custom_chart.save(path=f'../../charts/{yearmonth}', name=f'{today}_US_Fed_Rate')

WARN Conflicting scale property "domain" ([{"signal":"{data: datetime(\"2007-01-01T00:00:00\")}"},{"signal":"{data: datetime(\"2023-11-01T00:00:00\")}"}] and [["2007-01-01T00:00:00","2023-11-01T00:00:00"]]). Using the union of the two domains.
